In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tkinter as tk
from PIL import Image, ImageTk,ImageEnhance
from tkinter import filedialog,Toplevel
from tkinter import Menu

In [4]:
# Global variables to store the original and edited images
original_img = None
edited_img = None
current_r = 100
current_g = 100
current_b = 100
current_contrast = 100
exposure_value = 0
contrast_value = 0
shadow_value = 0
highlight_value = 0

# Function to open and display an image
def open_image():
    global original_img, edited_img,current_g,current_b,current_contrast,current_r, exposure_value, contrast_value, shadow_value, highlight_value, white_value, black_value
    current_r = 100
    current_g = 100
    current_b = 100
    current_contrast = 100
    # Reset adjustment values
    exposure_value = 0
    contrast_value = 0
    shadow_value = 0
    highlight_value = 0

    # Open a file dialog to select an image file
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.gif;*.tif")])
    if file_path:
        # Open the image
        original_img = Image.open(file_path)
        edited_img = original_img  # Copy original image as the initial edited image

        # Resize the images to fit the frames
        display_image(original_img, original_label)

# Function to display an image in a specific label
def display_image(image, label):
    resized_img = image.resize((450, 450))
    img_tk = ImageTk.PhotoImage(resized_img)
    label.config(image=img_tk)
    label.image = img_tk  # Keep a reference to avoid garbage collection


# Function to update the edited image based on RGB values from sliders
def update_color(r, g, b):
    global original_img, edited_img, current_r, current_g, current_b
    
    if edited_img:
        # Cập nhật giá trị RGB hiện tại
        
        # Điều chỉnh các kênh màu
        r_factor = r / current_r
        g_factor = g / current_g
        b_factor = b / current_b

        # Tách hình ảnh thành các kênh màu R, G, B
        r_img, g_img, b_img = edited_img.split()

        # Áp dụng các thay đổi dựa trên giá trị RGB đã lưu
        r_img = r_img.point(lambda i: i * r_factor)
        g_img = g_img.point(lambda i: i * g_factor)
        b_img = b_img.point(lambda i: i * b_factor)

        # Gộp các kênh màu và hiển thị
        edited_img = Image.merge("RGB", (r_img, g_img, b_img))
        current_r = r
        current_g = g
        current_b = b
        display_image(edited_img, edited_label)

# Function to open the color adjustment window
def open_color_window():
    color_window = Toplevel(root)
    color_window.title("Adjust Colors")
    color_window.geometry("300x300")

    # Variables to track RGB values from scales
    r_value = tk.IntVar(value=current_r)
    g_value = tk.IntVar(value=current_g)
    b_value = tk.IntVar(value=current_b)

    # Red Scale
    r_scale = tk.Scale(color_window, from_=10, to=200, orient="horizontal", label="Red", variable=r_value)
    r_scale.pack()
    # Green Scale
    g_scale = tk.Scale(color_window, from_=10, to=200, orient="horizontal", label="Green", variable=g_value)
    g_scale.pack()
    # Blue Scale
    b_scale = tk.Scale(color_window, from_=10, to=200, orient="horizontal", label="Blue", variable=b_value)
    b_scale.pack()
    
    # Update color when scale values change
    def on_scale_change(event=None):
        update_color(r_value.get(), g_value.get(), b_value.get())

    # Bind scale changes to color update
    r_scale.bind("<ButtonRelease-1>", on_scale_change)
    g_scale.bind("<ButtonRelease-1>", on_scale_change)
    b_scale.bind("<ButtonRelease-1>", on_scale_change)

def open_contrast_window():
    contrast_windown = Toplevel(root)
    contrast_windown.title("Contrast Window")
    contrast_windown.geometry("300x300")
    
    #Variable to keep track of contrast value
    contrast_value = tk.IntVar(value = current_contrast)
    
    # Contrast scale
    contrast_scale = tk.Scale(contrast_windown, from_=1, to = 200, orient="horizontal", label= "Contrast", variable= contrast_value)
    contrast_scale.pack()
    
    # Update color when scale values change
    def on_scale_change(event=None):
        update_contrast(contrast_value.get())

    # Bind scale changes to color update
    contrast_scale.bind("<ButtonRelease-1>", on_scale_change)
    
def update_contrast(contrast_value):
    global current_contrast,edited_img
    img_cv = np.array(edited_img)
    img_cv_abs = cv2.convertScaleAbs(img_cv, alpha= ( contrast_value/current_contrast), beta=0)
    edited_img = Image.fromarray(img_cv_abs)
    
    current_contrast = contrast_value
    
    display_image(edited_img,edited_label)
    #y = x * contrast_value; y' = y * contrast_value/current_contrast

# Create the main window
root = tk.Tk()
root.title("Scion Image")
root.geometry("1200x1200")

# Create the menu bar
menu_bar = Menu(root)

# Add "File" menu with "Open", "Edit Color", and "Exit" options
file_menu = Menu(menu_bar, tearoff=0)
file_menu.add_command(label="Open", command=open_image)

file_menu.add_command(label="Exit", command=root.quit)
menu_bar.add_cascade(label="File", menu=file_menu)

process_menu = Menu(menu_bar,tearoff=0)
process_menu.add_command(label="Edit Color", command=open_color_window)
process_menu.add_command(label="Contrast", command=open_contrast_window)
process_menu.add_command(label="Histogram", command=open_histogram_window)
menu_bar.add_cascade(label="Process",menu=process_menu)

root.config(menu=menu_bar)

# Create frames for original and edited images
frame_original = tk.Frame(root, width=300, height=400, bg="white")
frame_original.pack(side="left", fill="both", expand=True)

frame_edited = tk.Frame(root, width=300, height=400, bg="white")
frame_edited.pack(side="right", fill="both", expand=True)

# Labels to display images in each frame
original_label = tk.Label(frame_original, bg="white")
original_label.place(x=100, y=100)

# Label to display text "Original Image" below the original image
original_text_label = tk.Label(frame_original, text="Original Image", bg="white", font=("Arial", 10))
original_text_label.place(x=275, y=600)

edited_label = tk.Label(frame_edited, bg="white")
edited_label.place(x=100, y=100)

# Label to display text "Edited Image" below the edited image
edited_text_label = tk.Label(frame_edited, text="Edited Image", bg="white", font=("Arial", 10))
edited_text_label.place(x=275, y=600)

# Start the main loop
root.mainloop()


In [2]:
import cv2
import numpy as np
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import Toplevel

def is_gray_scale(image):
    # If the image is already in grayscale mode, it's grayscale
    if image.mode == "L":
        return True

    # If the image has an alpha channel (e.g., RGBA), ignore it
    if image.mode == "RGBA":
        image = image.convert("RGB")

    w, h = image.size
    for i in range(w):
        for j in range(h):
            pixel = image.getpixel((i, j))
            # Handle RGB and grayscale images
            if isinstance(pixel, tuple) and len(pixel) == 3:
                r, g, b = pixel
                if r != g or g != b:
                    return False
            else:
                # If it's not a tuple, assume grayscale
                return True
    return True

def open_histogram_window():
    global exposure_value, contrast_value, shadow_value, highlight_value, baseline_image

    # Create a new histogram window
    histogram_window = Toplevel(root)
    histogram_window.title("Histogram Window")
    histogram_window.geometry("600x700")

    baseline_image = edited_img.copy() if edited_img is not None else original_img.copy()

    def update_baseline_image():
        global baseline_image
        baseline_image = edited_img.copy() if edited_img is not None else original_img.copy()

    # Extract histogram data from an image
    def get_histogram_data(image):
        if image.mode == 'RGB' and not is_gray_scale(baseline_image):
            r, g, b = image.split()
            r_histogram = r.histogram()[:256]
            g_histogram = g.histogram()[:256]
            b_histogram = b.histogram()[:256]
            return r_histogram, g_histogram, b_histogram
        else:
            grayscale_img = image.convert("L")
            histogram = grayscale_img.histogram()[:256]
            return histogram, histogram, histogram

    bins = 256
    bar_width = 2
    canvas_width = bins * bar_width
    canvas_height = 200

    histogram_canvas = tk.Canvas(
        histogram_window,
        width=canvas_width,
        height=canvas_height,
        bg="white",
        highlightthickness=1,
        relief="solid"
    )
    histogram_canvas.pack(pady=10)

    def draw_histogram(data):
        histogram_canvas.delete("all")
        max_height = 200  # Set a static height for the histogram bars

        total_pixels = baseline_image.width * baseline_image.height
        
        if is_gray_scale(baseline_image):  # Grayscale image
            # max_value = max(data[0])
            channel_colors = ["black"]
            channel_offsets = [0]
        elif baseline_image.mode == 'RGB' and not is_gray_scale(baseline_image):  # RGB image
            # max_value = max(max(data[0]), max(data[1]), max(data[2]))
            channel_colors = ["red", "green", "blue"]
            channel_offsets = [-1, 0, 1]
        
        for channel in range(len(channel_colors)):  
            for i in range(bins):
                # bar_height = int(data[channel][i] / max_value * max_height)
                bar_height = int(data[channel][i] / (total_pixels/25) * max_height)
                x0 = i * bar_width + channel_offsets[channel]
                y0 = max_height - bar_height
                x1 = x0 + bar_width - 2
                y1 = max_height
                histogram_canvas.create_rectangle(x0, y0, x1, y1, fill=channel_colors[channel], outline="")

    def adjust_highlights(img_array, highlight_value):
        grayscale_img = baseline_image.convert("L")
        hist = grayscale_img.histogram()[:256]
        brightest_peak = np.argmax(hist[127:]) + 200

        highlight_range = 20
        lower_bound = max(0, brightest_peak - highlight_range)

        mask = (img_array >= lower_bound)

        max_value = max(255, np.max(img_array))
        min_value = min(0, np.min(img_array))

        if baseline_image.mode == 'RGB' and not is_gray_scale(baseline_image):
            # Handle RGB images
            for channel in range(3): 
                channel_data = img_array[:, :, channel]
                channel_data[mask[:, :, channel]] += highlight_value/4
                img_array[:, :, channel] = np.clip(channel_data, 0, 255)

                adjusted_values = channel_data[mask[:, :, channel]]
                mask_min_value, mask_max_value = None, None

                if adjusted_values.size > 0:
                    mask_min_value = adjusted_values.min()
                    mask_max_value = adjusted_values.max()
                if mask_min_value is not None or mask_max_value is not None:
                    if highlight_value > 0:
                        stretched_values = ((adjusted_values - mask_min_value) / (mask_max_value - mask_min_value)) * (255 - lower_bound) + lower_bound
                        channel_data[mask[:, :, channel]] = np.clip(stretched_values, 0, 255)
                    elif highlight_value < 0:
                        compressed_values = ((img_array[:, :, channel] - min_value) / (max_value - min_value)) * 255  
                        channel_data = np.clip(compressed_values, 0, 255)
                img_array[:, :, channel] = np.clip(channel_data, 0, 255)
        else:
            # Handle grayscale images
            img_array[mask] += highlight_value / 4
            img_array = np.clip(img_array, 0, 255)

            # Calculate min and max for the mask
            adjusted_values = img_array[mask]
            mask_min_value, mask_max_value = None, None

            if adjusted_values.size > 0:
                mask_min_value = adjusted_values.min()
                mask_max_value = adjusted_values.max()

            if mask_min_value is not None or mask_max_value is not None:
                if highlight_value > 0:
                    stretched_values = ((adjusted_values - mask_min_value) / (mask_max_value - mask_min_value)) * (255 - lower_bound) + lower_bound
                    img_array[mask] = np.clip(stretched_values, 0, 255)
                elif highlight_value < 0:
                    compressed_values = ((img_array - min_value) / (max_value - min_value)) * 255
                    img_array = np.clip(compressed_values, 0, 255)

        img_array = np.clip(img_array, 0, 255).astype(np.uint8)

        return img_array
    
    def adjust_shadows(img_array, shadow_value):
        img_array = img_array.astype(np.float64)

        grayscale_img = baseline_image.convert("L")
        hist = grayscale_img.histogram()[:256]
        brightest_peak = np.argmax(hist[127:]) + 127

        shadow_range = 20
        lower_bound = max(0, brightest_peak - shadow_range)

        mask = (img_array < lower_bound)

        max_value = np.max(img_array)
        min_value = np.min(img_array)

        if baseline_image.mode == 'RGB' and not is_gray_scale(baseline_image):
            # Handle RGB images
            for channel in range(3):
                channel_data = img_array[:, :, channel]
                channel_data[mask[:, :, channel]] += shadow_value / 4
                img_array[:, :, channel] = np.clip(channel_data, 0, 255)

                adjusted_values = channel_data[mask[:, :, channel]]
                mask_min_value, mask_max_value = None, None

                if adjusted_values.size > 0:
                    mask_min_value = adjusted_values.min()
                    mask_max_value = adjusted_values.max()

                if mask_min_value is not None or mask_max_value is not None:
                    if shadow_value > 0:
                        stretched_values = ((img_array[:, :, channel] - min_value) / (max_value - min_value)) * 255
                        channel_data = np.clip(stretched_values, 0, 255)
                    elif shadow_value < 0:
                        compressed_values = ((adjusted_values - mask_min_value) / (mask_max_value - mask_min_value)) * (lower_bound)
                        channel_data[mask[:, :, channel]] = np.clip(compressed_values, 0, 255)

                img_array[:, :, channel] = np.clip(channel_data, 0, 255)
        else:
            # Handle grayscale images
            img_array[mask] += shadow_value / 4
            img_array = np.clip(img_array, 0, 255)

            adjusted_values = img_array[mask]
            mask_min_value, mask_max_value = None, None

            if adjusted_values.size > 0:
                mask_min_value = adjusted_values.min()
                mask_max_value = adjusted_values.max()

            if mask_min_value is not None or mask_max_value is not None:
                if shadow_value > 0:
                    stretched_values = ((img_array - min_value) / (max_value - min_value)) * 255
                    img_array = np.clip(stretched_values, 0, 255)
                elif shadow_value < 0:
                    compressed_values = ((adjusted_values - mask_min_value) / (mask_max_value - mask_min_value)) * (lower_bound)
                    img_array[mask] = compressed_values

            img_array = np.clip(img_array, 0, 255).astype(np.uint8)

        return img_array


    def apply_adjustments():
        global exposure_value, contrast_value, shadow_value, highlight_value

        img_array = np.array(baseline_image).astype(np.float32)

        # exposure
        img_array += exposure_value

        # contranst
        if baseline_image.mode == 'RGB' and not is_gray_scale(baseline_image):
            # For RGB images
            mean_r = np.mean(img_array[:, :, 0])
            mean_g = np.mean(img_array[:, :, 1])
            mean_b = np.mean(img_array[:, :, 2])

            contrast_scale = 1 + (contrast_value / 170.0)

            img_array[:, :, 0] = (img_array[:, :, 0] - mean_r) * contrast_scale + mean_r
            img_array[:, :, 1] = (img_array[:, :, 1] - mean_g) * contrast_scale + mean_g
            img_array[:, :, 2] = (img_array[:, :, 2] - mean_b) * contrast_scale + mean_b

        else:
            # For grayscale images
            mean = np.mean(img_array)
            contrast_scale = 1 + (contrast_value / 170.0)
            img_array = (img_array - mean) * contrast_scale + mean

        # highlights 
        img_array = adjust_highlights(img_array, highlight_value)

        # shadows
        img_array = adjust_shadows(img_array, shadow_value)
        
        # Clip and convert to uint8
        img_array = np.clip(img_array, 0, 255).astype(np.uint8)

        # Update `edited_img`
        edited_img = Image.fromarray(img_array)

        # Update histogram
        display_image(edited_img, edited_label)
        updated_histogram_data = get_histogram_data(edited_img)
        draw_histogram(updated_histogram_data)

    # Sliders for adjusting each aspect
    def on_exposure_slider_change(event=None):
        global exposure_value
        exposure_value = exposure_slider.get()
        apply_adjustments()

    def on_contrast_slider_change(event=None):
        global contrast_value
        contrast_value = contrast_slider.get()
        apply_adjustments()

    def on_highlights_slider_change(event=None):
        global highlight_value
        highlight_value = highlights_slider.get()
        apply_adjustments()

    def on_shadows_slider_change(event=None):
        global shadow_value
        shadow_value = shadows_slider.get()
        apply_adjustments()

    # Exposure slider
    exposure_slider = tk.Scale(
        histogram_window,
        from_=-255,
        to=255,
        orient="horizontal",
        label="Exposure",
        command=on_exposure_slider_change
    )
    exposure_slider.set(exposure_value)
    exposure_slider.pack(pady=10)

    # Contrast slider
    contrast_slider = tk.Scale(
        histogram_window,
        from_=-100,
        to=100,
        orient="horizontal",
        label="Contrast",
        command=on_contrast_slider_change
    )
    contrast_slider.set(contrast_value)
    contrast_slider.pack(pady=10)

    # Highlights sliders
    highlights_slider = tk.Scale(
        histogram_window,
        from_=-100,
        to=100,
        orient="horizontal",
        label="Highlights",
        command=on_highlights_slider_change
    )
    highlights_slider.set(highlight_value)
    highlights_slider.pack(pady=10)

    # Shadows sliders
    shadows_slider = tk.Scale(
        histogram_window,
        from_=-100,
        to=100,
        orient="horizontal",
        label="Shadows",
        command=on_shadows_slider_change
    )
    shadows_slider.set(shadow_value)
    shadows_slider.pack(pady=10)

    # Equalize Histogram button
    def equalize_histogram():
        global edited_img, exposure_value, contrast_value, shadow_value, highlight_value, baseline_image
        
        exposure_value = 0
        contrast_value = 0
        shadow_value = 0
        highlight_value = 0
        exposure_slider.set(exposure_value)
        contrast_slider.set(contrast_value)
        highlights_slider.set(highlight_value)
        shadows_slider.set(shadow_value)

        if baseline_image.mode == 'RGB' and not is_gray_scale(baseline_image):
            img_array = np.array(baseline_image)
            for channel in range(3):
                img_array[:, :, channel] = cv2.equalizeHist(img_array[:, :, channel])
            edited_img = Image.fromarray(img_array)
        else:
            grayscale_img = cv2.cvtColor(np.array(baseline_image), cv2.COLOR_RGB2GRAY)
            equalized_img = cv2.equalizeHist(grayscale_img)
            edited_img = Image.fromarray(cv2.cvtColor(equalized_img, cv2.COLOR_GRAY2RGB))

        display_image(edited_img, edited_label)
        update_baseline_image()
        updated_histogram_data = get_histogram_data(edited_img)
        draw_histogram(updated_histogram_data)
        print("Histogram equalization applied to edited_img.")

    equalize_button = tk.Button(histogram_window, text="Equalize Histogram", command=equalize_histogram)
    equalize_button.pack(pady=10)

    # Initial histogram drawing
    original_histogram_data = get_histogram_data(baseline_image)
    draw_histogram(original_histogram_data)